In [1]:
!pip install outlines

You should consider upgrading via the 'E:\kaggle_comps\predict_future_sales\code\venv\new-venv\Scripts\python.exe -m pip install --upgrade pip' command.


## Imports

In [2]:
%load_ext autoreload
%autoreload 2

# Add the path to the parent directory to sys
import sys, os

# If current directory is called 'notebooks', chdir to the parent
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('../')
    
sys.path.append('attribution')

from torch.utils.data import DataLoader

import pandas as pd
from constants import ModelNames, DatasetNames, LANGUAGE_MAPPING
from model_utils import Model 
from dataset_utils import GSMDataset, PaddingCollator, is_correct_gsm, extract_answer_gsm

e:\kaggle_comps\predict_future_sales\code\venv\new-venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Necessary Codes

In [3]:
def load_model_dataset():
    model = Model(ModelNames.QwenInstruct)
    
    return model, DatasetNames.MGSM

## Analysis: Processing Responses

In [4]:
class ResponseProcessing():
    def __init__(self, model, dataset):
        self.df_column_names = ["question", "actual_answer", "model_gen_steps", "model_gen_answer" ]
        self.model = model
        self.dataset = dataset
    
    def convert_dashes_incremental_steps_list(self, step):
        furnished_steps = []

        i = 1
        for _, step in enumerate(steps[1:]):
            if step:  # Skip empty parts (if any)
                furnished_steps.append(str(i) + ". " + step )  # Replace with number (1, 2, 3...)
                i += 1
        
        return furnished_steps

    def convert_dashes_incremental_steps(self, step):

        '''
        Returns str
        '''

        furnished_steps = self.convert_dashes_incremental_steps_list(step)

        return "".join(furnished_steps)


    def process_model_responses_for_analysis(self):

        mgsm_test = GSMDataset(self.dataset, self.model.tokenizer, instructions='', split='test', config='en')

        mgsm_generation_df = pd.read_csv('results\mgsm_en_Qwen2.5-1.5B-Instruct_results.csv')
        mgsm_generations = mgsm_generation_df['response'].tolist()

        all_steps = []
        all_gen_final_ans = []

        for response in mgsm_generations:
            
            steps = response.split("\n-")
            final_step = steps[-1].split(".\r\n")[0]
            steps.pop()
            steps.append( final_step )

            steps_str = self.convert_dashes_incremental_steps(steps)
            all_steps.append( steps_str )

            gen_final_ans = extract_answer_gsm(response)
            all_gen_final_ans.append( gen_final_ans )

        question_list = mgsm_test.dataset['question']
        actual_answer = mgsm_test.dataset['answer_number']

        percentile_list = pd.DataFrame(data=zip(question_list,actual_answer,all_steps, all_gen_final_ans), columns=self.df_column_names)

        percentile_list.to_csv("results/analysis_mgsm_en_Qwen2.5-1.5B-Instruct_results.csv", index = False)



## Main

In [ ]:
if '__main__':
    model, dataset = load_model_dataset()
    
    responseProcessing = ResponseProcessing(model, dataset)
    responseProcessing.process_model_responses_for_analysis()

Error while downloading from https://cdn-lfs-us-1.hf.co/repos/ac/b2/acb26770a261dcc4f0e397d81ddea9e06b330d79f58c385663acdae527a7230e/dd924a11b4c220f385b51ffa522daea7c9f3d850e31b162bb5661df483c6d3ee?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1742837496&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MjgzNzQ5Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2FjL2IyL2FjYjI2NzcwYTI2MWRjYzRmMGUzOTdkODFkZGVhOWUwNmIzMzBkNzlmNThjMzg1NjYzYWNkYWU1MjdhNzIzMGUvZGQ5MjRhMTFiNGMyMjBmMzg1YjUxZmZhNTIyZGFlYTdjOWYzZDg1MGUzMWIxNjJiYjU2NjFkZjQ4M2M2ZDNlZT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=pUJfPNljM675iDfeAsVvUqRKrsw5PPdgOqotFgvoSP8dfCzeLW%7ESCF3PvMT0fDrqJDrQ169oq97tD4DnOgB1J-%7EJxeyBrBxHYDIi7Zggi2orHYQg-QszP264qL8FVGzXNKv--72TQwc38CiVHF7Yqo2gRQzEk-ZArqINFCmyEzWtBWIPojaU9W38quufiY8q6TYhikS79vTWa1eU5w1fFeh6BLRt2UTMDNNVABNk942MraCy7NW3MatjHfdlPgRk